In [3]:
import pandas as pd


In [84]:
df = pd.read_csv("Books_df.csv")

In [87]:
df.columns = df.columns.str.replace(" ","_").str.lower().str.replace("no.","num").str.replace(":","")

In [88]:
df

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls
0,0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock...
1,1,Black Holes (L) : The Reith Lectures [Paperbac...,Stephen Hawking,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹99.00,4.5,7686.0,https://www.amazon.in/Black-Holes-Lectures-Ste...
2,2,The Kite Runner,Khaled Hosseini,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹175.75,4.6,50016.0,https://www.amazon.in/Kite-Runner-Khaled-Hosse...
3,3,Greenlights: Raucous stories and outlaw wisdom...,Matthew McConaughey,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹389.00,4.6,32040.0,https://www.amazon.in/Greenlights-Raucous-stor...
4,4,The Science of Storytelling: Why Stories Make ...,Will Storr,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹348.16,4.5,1707.0,https://www.amazon.in/Science-Storytelling-Wil...
...,...,...,...,...,...,...,...,...,...,...
7923,7923,Insight Guides Poland (Travel Guide with Free ...,Insight Travel Guide,Travel,Travel & Holiday Guides,Paperback,"₹1,326.00",4.7,16.0,https://www.amazon.in/Insight-Guides-Poland-Tr...
7924,7924,Lonely Planet India 19 (Travel Guide),Anirban Mahapatra,Travel,Travel & Holiday Guides,Paperback,₹850.00,4.4,187.0,https://www.amazon.in/Lonely-Planet-India-Trav...
7925,7925,Eyewitness Travel Phrase Book French (EW Trave...,DK,Travel,Travel & Holiday Guides,Paperback,₹307.00,4.5,168.0,https://www.amazon.in/Eyewitness-Travel-Phrase...
7926,7926,Lonely Planet Australia (Travel Guide),Andrew Bain,Travel,Travel & Holiday Guides,Kindle Edition,"₹1,814.50",4.7,267.0,https://www.amazon.in/Lonely-Planet-Australia-...


In [90]:
df.dtypes

unnamed_0                int64
title                   object
author                  object
main_genre              object
sub_genre               object
type                    object
price                   object
rating                 float64
num_of_people_rated    float64
urls                    object
dtype: object

In [93]:
df['main_genre'].value_counts()

main_genre
Children's Books                         1450
Romance                                  1124
Sports                                    854
Literature & Fiction                      400
Computing, Internet & Digital Media       350
Teen & Young Adult                        350
Religion                                  300
Crafts, Home & Lifestyle                  300
Medicine & Health Sciences                250
Arts, Film & Photography                  200
Society & Social Sciences                 200
Reference                                 200
Higher Education Textbooks                200
Exam Preparation                          200
Sciences, Technology & Medicine           200
Politics                                  150
Business & Economics                      150
Science & Mathematics                     150
History                                   100
Health, Family & Personal Development     100
Biographies, Diaries & True Accounts      100
Fantasy, Horror & Scien

In [26]:
import requests

def buscar_libros_por_titulo(titulo):
    """ Busca libros en Open Library usando el título. """
    base_url = "https://openlibrary.org/search.json"
    params = {'title': titulo}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        libros = data.get('docs', [])
        resultados = []
        for libro in libros:
            resultados.append({
                'titulo': libro.get('title'),
                'autor': libro.get('author_name', ['Desconocido'])[0],
                'isbn': libro.get('isbn', [None])[0],
                'año_publicacion': libro.get('publish_year', [None])[0]
            })
        return resultados
    else:
        return f"Error en la búsqueda: código de estado {response.status_code}"

# Ejemplo de uso
titulo = "The Kite Runner Khaled Hosseini"  # Puedes cambiar este título por cualquier otro
resultados_libros = buscar_libros_por_titulo(titulo)
for libro in resultados_libros:
    print(libro)

# STEP 1 : recorrer todo el data set y buscar con esta API el ISBN de cada libro y el año de publicación


# STEP 2 : guardar solo si el titulo es similar al titulo del libro en el data set, en 80% de similitud

{'titulo': 'The Kite Runner, Khaled Hosseini', 'autor': 'Spark Publishing', 'isbn': '1411470990', 'año_publicacion': 2014}
{'titulo': "Khaled Hosseini's The kite runner", 'autor': 'Harold Bloom', 'isbn': '9781604131994', 'año_publicacion': 2009}
{'titulo': 'Complete Khaled Hosseini (And the Mountains Echoed / Kite Runner / Thousand Splendid Suns))', 'autor': 'Khaled Hosseini', 'isbn': '9781408868195', 'año_publicacion': 2015}
{'titulo': 'The Kite Runner by Khaled Hosseini', 'autor': 'Bright Summaries', 'isbn': '9782806271150', 'año_publicacion': 2015}
{'titulo': 'By Hosseini, Khaled The Kite Runner Paperback', 'autor': 'Khaled Hosseini', 'isbn': None, 'año_publicacion': 2004}
{'titulo': "Study Guide for Khaled Hosseini's the Kite Runner", 'autor': 'Course Hero', 'isbn': '9798516175275', 'año_publicacion': 2021}
{'titulo': 'Study Guide to the Kite Runner by Khaled Hosseini', 'autor': 'Ray Moore', 'isbn': '9798667371328', 'año_publicacion': 2020}
{'titulo': 'Bookclub-In-A-Box Discusses t

In [ ]:
# if resultados_libros[0]["titulo"]  sea igual en 80% titulo: 

#Si verdad entonces guardar en un nuevo data set( isbn y el año de publicación)

In [69]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
"""def titulos_con_similitud()"""
def calcular_similitud(texto1, texto2):
    vectorizer = TfidfVectorizer()
    
    # Transforma los textos en vectores TF-IDF
    tfidf = vectorizer.fit_transform([texto1, texto2])
    
    # Calcula la similitud del coseno entre los dos vectores
    similitud = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
    return similitud

def buscar_libros_por_titulo_con_similitud(titulo):
    
    base_url = "https://openlibrary.org/search.json"
    params = {'title': titulo}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        libros = data.get('docs', [])
        resultados = []
        for libro in libros:
            resultados.append({
            'titulo': libro.get('title'),
            'autor': libro.get('author_name', ['Desconocido'])[0],
            'isbn': libro.get('isbn', [None])[0],
            'año_publicacion': libro.get('publish_year', [None])[0]
            })
        return resultados
    else:
        return f"Error en la búsqueda: código de estado {response.status_code}"

    resultados = buscar_libros_por_titulo(titulo)
    resultados_con_similitud = []

    for libro in resultados:
        similitud = calcular_similitud(titulo, libro['titulo'])
        if similitud >= 0.80:
            libro['similitud'] = similitud
            resultados_con_similitud.append(libro)

    return resultados_con_similitud

In [94]:
df.head()

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls
0,0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock...
1,1,Black Holes (L) : The Reith Lectures [Paperbac...,Stephen Hawking,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹99.00,4.5,7686.0,https://www.amazon.in/Black-Holes-Lectures-Ste...
2,2,The Kite Runner,Khaled Hosseini,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹175.75,4.6,50016.0,https://www.amazon.in/Kite-Runner-Khaled-Hosse...
3,3,Greenlights: Raucous stories and outlaw wisdom...,Matthew McConaughey,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹389.00,4.6,32040.0,https://www.amazon.in/Greenlights-Raucous-stor...
4,4,The Science of Storytelling: Why Stories Make ...,Will Storr,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹348.16,4.5,1707.0,https://www.amazon.in/Science-Storytelling-Wil...


In [61]:
base_url = "https://openlibrary.org/search.json"
params = {'title': titulo}
response = requests.get(base_url, params=params)
if response.status_code == 200:
    data = response.json()
    libros = data.get('docs', [])
    resultados = []

In [95]:
df["title"][0]

'The Complete Novel of Sherlock Holmes'

In [98]:
df.head(2)

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls
0,0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock...
1,1,Black Holes (L) : The Reith Lectures [Paperbac...,Stephen Hawking,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹99.00,4.5,7686.0,https://www.amazon.in/Black-Holes-Lectures-Ste...


In [105]:
buscar_libros_por_titulo_con_similitud(df["title"][1])

[]

In [101]:
buscar_libros_por_titulo_con_similitud(df["title"][0])[0]['autor']

'Arthur Conan Doyle'

In [ ]:
for i in range (0,len(df)):
    autor = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['autor']
    df.loc[i, 'autor_api'] = autor
    year_public = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['año_publicacion']
    df.loc[i, 'year_public'] = year_public
    isbn = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['isbn']
    df.loc[i, 'isbn'] = isbn

In [107]:

for i in range (0,len(df)):
    try:
        autor = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['autor']
        df.loc[i, 'autor_api'] = autor
    
        year_public = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['año_publicacion']
        df.loc[i, 'year_public'] = year_public

        isbn = buscar_libros_por_titulo_con_similitud(df["title"][i])[0]['isbn']
        df.loc[i, 'isbn'] = isbn
    except:
        None


In [108]:
df

,unnamed_0,title,author,main_genre,sub_genre,type,price,rating,num_of_people_rated,urls,autor_api,year_public,isbn
0,0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock...,Arthur Conan Doyle,2017.0,8175994312
1,1,Black Holes (L) : The Reith Lectures [Paperbac...,Stephen Hawking,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹99.00,4.5,7686.0,https://www.amazon.in/Black-Holes-Lectures-Ste...,NaN,NaN,NaN
2,2,The Kite Runner,Khaled Hosseini,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹175.75,4.6,50016.0,https://www.amazon.in/Kite-Runner-Khaled-Hosse...,Khaled Hosseini,2016.0,0756948800
3,3,Greenlights: Raucous stories and outlaw wisdom...,Matthew McConaughey,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹389.00,4.6,32040.0,https://www.amazon.in/Greenlights-Raucous-stor...,NaN,NaN,NaN
4,4,The Science of Storytelling: Why Stories Make ...,Will Storr,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹348.16,4.5,1707.0,https://www.amazon.in/Science-Storytelling-Wil...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7923,7923,Insight Guides Poland (Travel Guide with Free ...,Insight Travel Guide,Travel,Travel & Holiday Guides,Paperback,"₹1,326.00",4.7,16.0,https://www.amazon.in/Insight-Guides-Poland-Tr...,NaN,NaN,NaN
7924,7924,Lonely Planet India 19 (Travel Guide),Anirban Mahapatra,Travel,Travel & Holiday Guides,Paperback,₹850.00,4.4,187.0,https://www.amazon.in/Lonely-Planet-India-Trav...,NaN,NaN,NaN
7925,7925,Eyewitness Travel Phrase Book French (EW Trave...,DK,Travel,Travel & Holiday Guides,Paperback,₹307.00,4.5,168.0,https://www.amazon.in/Eyewitness-Travel-Phrase...,NaN,NaN,NaN
7926,7926,Lonely Planet Australia (Travel Guide),Andrew Bain,Travel,Travel & Holiday Guides,Kindle Edition,"₹1,814.50",4.7,267.0,https://www.amazon.in/Lonely-Planet-Australia-...,NaN,NaN,NaN


In [102]:
len(df)

7928

In [63]:
for libro in libros:
            resultados.append({
            'titulo': libro.get('title'),
            'autor': libro.get('author_name', ['Desconocido'])[0],
            'isbn': libro.get('isbn', [None])[0],
            'año_publicacion': libro.get('publish_year', [None])[0]
            })

In [67]:
resultados = buscar_libros_por_titulo(titulo)
resultados_con_similitud = []
for libro in resultados:
    similitud = calcular_similitud(titulo, libro['title'])
    if similitud >= 0.80:
        libro['similitud'] = similitud
        resultados_con_similitud.append(libro)

In [73]:
resultados_con_similitud

[]

In [71]:
titulo ="The Complete Novel of Sherlock Holmes" 
buscar_libros_por_titulo_con_similitud(titulo)

[{'titulo': 'The Complete Novels of Sherlock Holmes (Adventures of Sherlock Holmes / Hound of the Baskervilles / Memoirs of Sherlock Holmes / Return of Sherlock Holmes / Sign of Four / Study in Scarlet / Valley of Fear)',
  'autor': 'Arthur Conan Doyle',
  'isbn': '8175994312',
  'año_publicacion': 2017}]

In [72]:
texto1 = df["Title"][0]
texto2 = titulo 
calcular_similitud(texto1, texto2)
buscar_libros_por_titulo_con_similitud(titulo)

[{'titulo': 'The Complete Novels of Sherlock Holmes (Adventures of Sherlock Holmes / Hound of the Baskervilles / Memoirs of Sherlock Holmes / Return of Sherlock Holmes / Sign of Four / Study in Scarlet / Valley of Fear)',
  'autor': 'Arthur Conan Doyle',
  'isbn': '8175994312',
  'año_publicacion': 2017}]

In [45]:
#Calcular la similitud :

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calcular_similitud(texto1, texto2):
    vectorizer = TfidfVectorizer()
    
    # Transforma los textos en vectores TF-IDF
    tfidf = vectorizer.fit_transform([texto1, texto2])
    
    # Calcula la similitud del coseno entre los dos vectores
    similitud = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
    
    return similitud

# Ejemplo de uso
texto1 = "Este es un ejemplo de texto para comparar."
texto2 = "Este es un texto de ejemplo muy similar para comparar."

similitud = calcular_similitud(texto1, texto2)
print(f"La similitud del coseno entre los textos es: {similitud:.2f}")

# Comprobar si la similitud es al menos del 80%
if similitud >= 0.80:
    print("Los textos son similares en un 80% o más.")
else:
    print("Los textos no son lo suficientemente similares.")

La similitud del coseno entre los textos es: 0.82
Los textos son similares en un 80% o más.
